# Initialization
## Load libraries

In [ ]:
# ML libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from captum.attr import IntegratedGradients, DeepLift, GradientShap, NoiseTunnel, FeatureAblation
from captum.attr import LayerConductance

# Visualization
import plotly.graph_objects as go
import plotly.express as px

# Utilities
import pandas as pd
import numpy as np
import datetime
import pprint

from sklearn.preprocessing import MinMaxScaler

# Custom
from tool_box.model import DenseNN
from tool_box.utilities import pair_plot_sample

## Config

In [ ]:
# Set timestamp for run ID
run_timestamp = datetime.datetime.now()

torch.manual_seed(44)
pp = pprint.PrettyPrinter(width=41, compact=True)

# Load and prepare data
## Load data

In [ ]:
df = pd.read_csv('./source/INCART 2-lead Arrhythmia Database.csv')

## Redefine target

In [ ]:
df['anomaly_ind'] = 0
df.loc[df.type != 'N', 'anomaly_ind'] = 1
df = df.drop(['type'], axis=1)

## Remove non-used columns

In [ ]:
df = df.drop(['record'], axis=1)


# Exploratory analysis

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
any(df.isnull().sum() != 0)

In [ ]:
df.groupby('anomaly_ind').agg({'0_pre-RR':'count'}).rename(columns={'0_pre-RR':'Count'})

In [ ]:
df.describe()

In [ ]:
pp_df = pair_plot_sample(df, 'anomaly_ind', 1000)

pair_comb = (
    ('RR'),
    ('Peak'),
    ('interval'),
    ('morph0', 'morph1', 'morph2', 'morph3', 'morph4'),
)

for c in pair_comb:

    fig = px.scatter_matrix(
        pp_df,
        dimensions=[col for col in pp_df.columns if col.endswith(c)],
        title='Relation between metrics',
        color_continuous_scale='Bluered_r',
        color='anomaly_ind',
        opacity=0.05,
        width=1500, height=900
    )

    fig.show()

# Prepare data
## Create features and target datasets

In [ ]:
target = df.anomaly_ind.to_numpy()
features_names = list(df.columns)
features = df.drop(['anomaly_ind'], axis=1).to_numpy()

## Scale data

In [ ]:
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

## Splitting data

In [ ]:
TEST_FRAC = 0.3
samples = len(target)

train_indices = np.random.choice(samples, int((1 - TEST_FRAC)*samples), replace=False)
test_indices = list(set(range(samples)) - set(train_indices))

train_label, train_feature = target[train_indices], features[train_indices]
test_label, test_feature = target[test_indices], features[test_indices]

print(f'Training samples: {train_feature.shape[0]}\nTesting samples: {test_feature.shape[0]}')

## Transform data into tensors

In [ ]:
train_feature_tensor = torch.from_numpy(train_feature).type(torch.FloatTensor)
train_label_tensor = torch.from_numpy(train_label).type(torch.FloatTensor)

test_feature_tensor = torch.from_numpy(test_feature).type(torch.FloatTensor)
test_label_tensor = torch.from_numpy(test_label).type(torch.FloatTensor)

train_data = list(zip(train_feature_tensor, train_label_tensor))
test_data = list(zip(test_feature_tensor, test_label_tensor))

## Create Dataloader

In [ ]:
BATCH_SIZE = 2**9

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle = False)

# Model
## Create model

In [ ]:
dense_nn = DenseNN(
    train_feature_tensor.shape[1],
    train_feature_tensor.shape[1]*2+1,
    train_feature_tensor.shape[1]*1+1,
    1,
    criterion = nn.BCELoss(),
    optimizer = 'adam',
    learning_rate=0.01
)

### Run model

In [ ]:
train_loss_dict = dict()
test_loss_dict = dict()
num_epochs = 300

for epoch in range(num_epochs):

    train_loss_lst = list()
    test_loss_lst = list()

    for i, train_batch in enumerate(train_loader):

        train_loss = dense_nn.training_step(train_batch)
        train_loss_lst.append(train_loss)

    for i, test_batch in enumerate(test_loader):

        test_loss = dense_nn.testing_step(test_batch)
        test_loss_lst.append(test_loss)

    train_loss_dict[epoch] = train_loss_lst
    test_loss_dict[epoch] = test_loss_lst

    if epoch % 50 == 0:
        print ('Epoch {}/{} => Train loss: {:.3f} - Test loss: {:.3f}'.format(
            epoch+1,
            num_epochs,
            np.mean(train_loss_dict[epoch]),
            np.mean(test_loss_dict[epoch])
            )
        )

model_path = f"./models/dense_nn_{run_timestamp.strftime('%Y%m%d%H%M%S')}.pt"
torch.save(dense_nn.state_dict(), model_path)

## Results

In [ ]:
results = dense_nn.model_eval(train_feature_tensor, train_label_tensor, test_feature_tensor, test_label_tensor)
pp.pprint(results)

In [ ]:
sample_train_loss_lst = [item for sublist in train_loss_dict.values() for item in sublist[:len(test_loss_dict[0])]]
sample_test_loss_lst = [item for sublist in test_loss_dict.values() for item in sublist]

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=list(range(len(sample_train_loss_lst))), y=sample_train_loss_lst, name= 'Training', mode='lines', line_color='blue', opacity=1)
)
fig.add_trace(
    go.Scatter(x=list(range(len(sample_test_loss_lst))), y=sample_test_loss_lst, name= 'Testing', mode='lines', line_color='red', opacity=0.5)
)

fig.update_layout(
    title="Average loss by epoch",
    xaxis_title="Epoch",
    yaxis_title="Loss",
    font=dict(size=15),
    width=1300,
    height=600)

fig.show()

# Explainable IA
## Set up attribution algorithms

In [ ]:
sample_test_indices = torch.randperm(test_feature_tensor.size(0))[:3000]
sample_test_feature_tensor = test_feature_tensor[sample_test_indices]

In [ ]:
attr_algos = {
    'IntegratedGradients': dict(),
    'DeepLift': dict(),
    'FeatureAblation': dict()
}

for algo in attr_algos:
    attr_algos[algo]['algorithm'] = eval(f"{algo}(dense_nn)")
    attr_algos[algo]['attr'] = attr_algos[algo]['algorithm'].attribute(sample_test_feature_tensor)

## Feature importance

In [ ]:
x_labels = list(df.drop('anomaly_ind', axis=1).columns)

for algo in attr_algos:
    attr_algos[algo]['sum'] = attr_algos[algo]['attr'].detach().numpy().sum(axis=0)
    attr_algos[algo]['euc_norm'] = np.linalg.norm(attr_algos[algo]['sum'], ord=1)
    attr_algos[algo]['norm_vec'] = attr_algos[algo]['sum'] / attr_algos[algo]['euc_norm']

In [ ]:
fig = go.Figure()

for algo in attr_algos:
    fig.add_trace(
        go.Bar(x=x_labels, y=attr_algos.get(algo).get('norm_vec'), name=algo)
    )

fig.show()

## Prediction analysis

In [ ]:
attributions, approximation_error = attr_algos.\
    get('IntegratedGradients').\
    get('algorithm').\
    attribute(test_feature_tensor[0].reshape(1, -1), return_convergence_delta = True)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Bar(x=x_labels, y=attributions.numpy().reshape(-1,), name='IntegratedGradients')
)

fig.show()

In [ ]:
approximation_error

## Model analysis

In [ ]:
layer_2 = LayerConductance(dense_nn, dense_nn.z2)
layer_2_attr = layer_2.attribute(sample_test_feature_tensor, n_steps=50, attribute_to_layer_input=True)

In [ ]:
layer_2_gral_attr = layer_2_attr.mean(axis=0).detach().numpy()
layer_2_norm_attr = layer_2_gral_attr / np.linalg.norm(layer_2_gral_attr, ord=1)

layer_2_gral_weights = dense_nn.z2.weight.mean(axis=0).detach().numpy()
layer_2_norm_weights = layer_2_gral_weights / np.linalg.norm(layer_2_gral_weights, ord=1)

In [ ]:
neurons = ['Neuron ' + str(l+1) for l in range(len(layer_2_norm_weights))]
neurons_values = {
    'Weights':layer_2_norm_weights,
    'Attributions':layer_2_norm_attr
}

In [ ]:
fig = go.Figure()

for val in neurons_values:
    fig.add_trace(
        go.Bar(x=neurons, y=neurons_values.get(val), name=val)
    )

fig.show()